# Use HYPER_PERSNL_Template Customized Notebook Template

In [2]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
infer_datetime_format=True

import numpy as np
import random

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(9,7), 'figure.dpi':120})

random.seed(0)

now1 = datetime.now()

In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [4]:
table_name = 'CUSTOMER_ACCOUNT_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
account = sf_df.to_pandas()

In [ ]:
table_name = 'CUSTOMER_TRANSACTION_DETAILS'

sf_df = my_session.sql("select * from {}".format(table_name))
transactions = sf_df.to_pandas()


In [ ]:
table_name = '<TABLE NAME>'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [ ]:
table_name = '<TABLE NAME>'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [ ]:
table_name = '<TABLE NAME>'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [ ]:
table_name = '<TABLE NAME>'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()